In [3]:

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
import ta
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

import torch.nn as nn


In [ ]:
# Load and preprocess data
def load_data(file_path):
    df = pd.read_csv(file_path)
    # df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
    df['Datetime'] = pd.to_datetime(df['Date'])
    # df = df.sort_values('Datetime').drop(['Date', 'Time'], axis=1)
    df = df.sort_values('Datetime').drop(['Date'], axis=1)
    return df

def add_technical_indicators(df):
    """Add technical indicators."""
    print("Adding technical indicators...")
    # Momentum
    df['RSI'] = ta.momentum.RSIIndicator(df['Close']).rsi()
    df['Momentum'] = ta.momentum.ROCIndicator(df['Close']).roc()
    df['CMO'] = ta.momentum.kama(df['Close'])
    df['Williams_%R'] = ta.momentum.WilliamsRIndicator(df['High'], df['Low'], df['Close']).williams_r()
    # Volatility
    df['ATR'] = ta.volatility.AverageTrueRange(df['High'], df['Low'], df['Close']).average_true_range()
    bb = ta.volatility.BollingerBands(df['Close'])
    df['BB_Mid'] = bb.bollinger_mavg()
    df['BB_Upper'] = bb.bollinger_hband()
    df['BB_Lower'] = bb.bollinger_lband()
    df['BB_Bandwidth'] = bb.bollinger_wband()
    keltner = ta.volatility.KeltnerChannel(df['High'], df['Low'], df['Close'])
    df['KC_High'] = keltner.keltner_channel_hband()
    df['KC_Low'] = keltner.keltner_channel_lband()
    donchian = ta.volatility.DonchianChannel(df['High'], df['Low'], df['Close'])
    df['DC_High'] = donchian.donchian_channel_hband()
    df['DC_Low'] = donchian.donchian_channel_lband()
    # Trend
    df['SMA_20'] = ta.trend.SMAIndicator(df['Close'], window=20).sma_indicator()
    df['EMA_20'] = ta.trend.EMAIndicator(df['Close'], window=20).ema_indicator()
    df['DPO'] = ta.trend.DPOIndicator(df['Close']).dpo()
    macd = ta.trend.MACD(df['Close'])
    df['MACD'] = macd.macd()
    df['MACD_Hist'] = macd.macd_diff()
    df['Mass_Index'] = ta.trend.mass_index(df['High'], df['Low'])
    # Volume
    df['AD'] = ta.volume.AccDistIndexIndicator(df['High'], df['Low'], df['Close'], df['Volume']).acc_dist_index()
    df['CMF'] = ta.volume.ChaikinMoneyFlowIndicator(df['High'], df['Low'], df['Close'], df['Volume']).chaikin_money_flow()
    df['Force_Index'] = ta.volume.ForceIndexIndicator(df['Close'], df['Volume']).force_index()
    df['MFI'] = ta.volume.MFIIndicator(df['High'], df['Low'], df['Close'], df['Volume']).money_flow_index()
    df['OBV'] = ta.volume.OnBalanceVolumeIndicator(df['Close'], df['Volume']).on_balance_volume()

    print("Technical indicators added.")
    return df.reset_index(drop=True)

def add_basic_features(df):
    """Final preprocessing: pct_change, time features."""
    print("Applying preprocessing...")
    # Calculate percentage change for OHLC
    cols_to_pct = ['Open', 'High', 'Low', 'Close']
    existing_cols = [col for col in cols_to_pct if col in df.columns]
    if existing_cols:
        print(f"Calculating percentage change for: {existing_cols}")
        df[existing_cols] = df[existing_cols].pct_change().fillna(0) * 100
        df['Cum_Return'] = df['Close'].rolling(window=20).sum()
        df['Cum_Turnover'] = df['Volume'].rolling(window=20).sum()

    # Add time features
    if 'Datetime' in df.columns:
        print("Adding time features...")
        df['Hour'] = df['Datetime'].dt.hour / 23.0
        df['Day_Of_Week'] = df['Datetime'].dt.dayofweek / 6.0
        df['Minute_Of_Day'] = (df['Datetime'].dt.hour * 60 + df['Datetime'].dt.minute) / 1439.0

    print("Preprocessing completed.")
    return df.reset_index(drop=True)

def normalize_by_blocks(data, block_size):
    print(f"Applying normalize_by_blocks with block_size={block_size}...")
    if isinstance(data, pd.DataFrame):
        columns = data.columns
        index = data.index
        data_np = data.values.astype('float32')
    else:
        data_np = data.astype('float32')
        columns = None
        index = None

    result = np.zeros_like(data_np)
    num_blocks = 0

    for start_idx in range(0, len(data_np), block_size):
        end_idx = min(start_idx + block_size, len(data_np))
        block = data_np[start_idx:end_idx]

        if block.shape[0] > 0:
            scaler = StandardScaler()
            if block.shape[0] == 1:
                normalized_block = block - np.mean(block, axis=0)
            else:
                std_devs = np.std(block, axis=0)
                if np.any(std_devs == 0):
                    normalized_block = np.zeros_like(block)
                    valid_cols = std_devs != 0
                    if np.any(valid_cols):
                        scaler.fit(block[:, valid_cols])
                        normalized_block[:, valid_cols] = scaler.transform(block[:, valid_cols])
                    zero_std_cols = std_devs == 0
                    if np.any(zero_std_cols):
                        normalized_block[:, zero_std_cols] = block[:, zero_std_cols] - np.mean(block[:, zero_std_cols], axis=0)
                else:
                    normalized_block = scaler.fit_transform(block)

            if np.isnan(normalized_block).any() or np.isinf(normalized_block).any():
                normalized_block = np.nan_to_num(normalized_block, nan=0.0, posinf=0.0, neginf=0.0)

            result[start_idx:end_idx] = normalized_block
            num_blocks += 1

    print(f"normalize_by_blocks completed. Processed {num_blocks} blocks.")
    if columns is not None and index is not None:
        return pd.DataFrame(result, columns=columns, index=index)
    else:
        return result


In [4]:
# Model definition
class InceptionModule(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.branch1 = nn.Conv1d(in_channels, 32, kernel_size=1, padding='same')
        self.branch3 = nn.Conv1d(in_channels, 32, kernel_size=3, padding='same')
        self.branch5 = nn.Conv1d(in_channels, 32, kernel_size=5, padding='same')
        self.branch_pool = nn.Sequential(
            nn.MaxPool1d(kernel_size=3, stride=1, padding=1),
            nn.Conv1d(in_channels, 32, kernel_size=1)
        )

    def forward(self, x):
        return torch.cat([self.branch1(x), self.branch3(x), self.branch5(x), self.branch_pool(x)], dim=1)

class Time2Vec(nn.Module):
    def __init__(self, time_dim, kernel_dim=32):
        super().__init__()
        self.linear = nn.Linear(time_dim, 1)
        self.periodic = nn.Linear(time_dim, kernel_dim - 1)

    def forward(self, x):
        linear = self.linear(x)
        periodic = self.periodic(x)
        return torch.cat([linear, periodic], dim=-1)

class CrossAttentionFusion(nn.Module):
    def __init__(self, cnn_dim, transformer_dim):
        super().__init__()
        self.query = nn.Linear(cnn_dim, transformer_dim)
        self.key = nn.Linear(transformer_dim, transformer_dim)
        self.value = nn.Linear(transformer_dim, transformer_dim)
        
    def forward(self, cnn_features, transformer_features):
        Q = self.query(cnn_features).unsqueeze(1)  # [batch, 1, transformer_dim]
        K = self.key(transformer_features)         # [batch, seq_len, transformer_dim]
        V = self.value(transformer_features)       # [batch, seq_len, transformer_dim]
        
        attn_scores = (Q @ K.transpose(-2, -1)) / (K.size(-1) ** 0.5)  # [batch, 1, seq_len]
        attn_weights = torch.softmax(attn_scores, dim=-1)
        
        return torch.bmm(attn_weights, V).squeeze(1)  # [batch, transformer_dim]

class HighwayNetwork(nn.Module):
    def __init__(self, d_model):
        super().__init__()
        self.gate = nn.Sequential(
            nn.Linear(d_model, d_model),
            nn.Sigmoid()
        )
        
    def forward(self, fused, transformer):
        g = self.gate(fused)
        return g * fused + (1 - g) * transformer

class EnhancedHybridModel(nn.Module):
    def __init__(self, num_features, time_dim, num_classes=3, d_model=512, nhead=16, dim_feedforward=1024, num_layers=6):
        super().__init__()
        # 1. InceptionTime Branch
        self.inception = nn.Sequential(
            InceptionModule(num_features),
            nn.ReLU(),
            nn.MaxPool1d(2),
            InceptionModule(128),
            nn.ReLU()
        )
        
        # 2. Transformer Branch
        self.time2vec = Time2Vec(time_dim, d_model//2)
        self.transformer_proj = nn.Linear(num_features, d_model//2)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward, batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        # 3. Fusion
        self.cross_attention = CrossAttentionFusion(128, d_model)
        self.highway = HighwayNetwork(d_model)
        
        # 4. Classifier
        self.classifier = nn.Sequential(
            nn.Linear(d_model, 128),
            nn.LayerNorm(128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )

    def forward(self, x, time_feature):
        # 1. Inception Path
        cnn_features = self.inception(x.permute(0, 2, 1))  # [batch, channels, seq_len//2]
        cnn_features = cnn_features.mean(dim=-1)          # [batch, channels=128]
        
        # 2. Transformer Path
        time_embed = self.time2vec(time_feature)  # [batch, seq_len, d_model // 2]
        x_proj = self.transformer_proj(x)  # [batch, seq_len, d_model // 2]
        combined = torch.cat([time_embed, x_proj], dim=-1) # [batch, seq_len, d_model]
        transformer_features = self.transformer(combined)  # [batch, seq_len, d_model]
        
        # 3. Fusion
        fused = self.cross_attention(cnn_features, transformer_features)  # [batch, d_model]
        output = self.highway(fused, transformer_features.mean(dim=1))   # [batch, d_model]
        
        return self.classifier(output)


In [5]:
class BacktestDataset(Dataset):
    def __init__(self, df, sequence_length, feature_cols, use_time2vec=True):
        self.sequence_length = sequence_length
        self.feature_cols = [col for col in feature_cols if col in df.columns]
        self.use_time2vec = use_time2vec
        self.has_time_input = False

        if self.use_time2vec:
            if "Minute_Of_Day" in df.columns and pd.api.types.is_numeric_dtype(df['Minute_Of_Day']):
                self.time_features = df["Minute_Of_Day"].values[:, np.newaxis].astype(np.float32)
                self.has_time_input = True
            else:
                self.time_features = np.zeros((len(df), 1), dtype=np.float32)
        else:
            self.time_features = np.zeros((len(df), 1), dtype=np.float32)

        self.features = df[self.feature_cols].values.astype(np.float32)
        self.datetimes = df['Datetime'].values
        self.close_prices = df['Close'].values

        if len(self.features) <= self.sequence_length:
            raise ValueError(f"DataFrame length ({len(self.features)}) must be greater than sequence_length ({self.sequence_length}).")

    def __len__(self):
        return len(self.features) - self.sequence_length

    def __getitem__(self, idx):
        end_idx = idx + self.sequence_length
        feat_seq = self.features[idx:end_idx]
        time_seq = self.time_features[idx:end_idx]
        datetime = self.datetimes[end_idx]
        close_price = self.close_prices[end_idx]
        return torch.from_numpy(feat_seq).float(), torch.from_numpy(time_seq).float(), datetime, close_price

class TradingStrategy:
    def __init__(self, initial_balance=10000, position_size=0.1, stop_loss=0.02, take_profit=0.04):
        self.initial_balance = initial_balance
        self.balance = initial_balance
        self.position_size = position_size
        self.stop_loss = stop_loss
        self.take_profit = take_profit
        
        self.current_position = None
        self.entry_price = None
        self.trades = []
        self.equity_curve = []
        
    def reset(self):
        self.balance = self.initial_balance
        self.current_position = None
        self.entry_price = None
        self.trades = []
        self.equity_curve = []
        
    def open_position(self, position_type, price, datetime):
        if self.current_position is not None:
            return
            
        self.current_position = position_type
        self.entry_price = price
        position_value = self.balance * self.position_size
        units = position_value / price
        
        self.trades.append({
            'datetime': datetime,
            'type': 'ENTRY',
            'position': position_type,
            'price': price,
            'units': units,
            'balance': self.balance
        })
        
    def close_position(self, price, datetime, reason='EXIT'):
        if self.current_position is None:
            return
            
        last_trade = self.trades[-1]
        units = last_trade['units']
        
        if self.current_position == 'BUY':
            pnl = (price - self.entry_price) * units
        else:  # SELL
            pnl = (self.entry_price - price) * units
            
        self.balance += pnl
        
        self.trades.append({
            'datetime': datetime,
            'type': reason,
            'position': self.current_position,
            'price': price,
            'units': units,
            'pnl': pnl,
            'balance': self.balance
        })
        
        self.current_position = None
        self.entry_price = None
        
    def check_stop_loss_take_profit(self, current_price, datetime):
        if self.current_position is None:
            return
            
        price_change = (current_price - self.entry_price) / self.entry_price
        
        if self.current_position == 'BUY':
            if price_change <= -self.stop_loss:
                self.close_position(current_price, datetime, 'STOP_LOSS')
            elif price_change >= self.take_profit:
                self.close_position(current_price, datetime, 'TAKE_PROFIT')
        else:  # SELL
            if price_change >= self.stop_loss:
                self.close_position(current_price, datetime, 'STOP_LOSS')
            elif price_change <= -self.take_profit:
                self.close_position(current_price, datetime, 'TAKE_PROFIT')
                
    def update_equity(self, current_price, datetime):
        equity = self.balance
        
        if self.current_position is not None:
            last_trade = self.trades[-1]
            units = last_trade['units']
            
            if self.current_position == 'BUY':
                pnl = (current_price - self.entry_price) * units
            else:  # SELL
                pnl = (self.entry_price - current_price) * units
                
            equity += pnl
            
        self.equity_curve.append({
            'datetime': datetime,
            'equity': equity
        })
        
    def get_trades_df(self):
        return pd.DataFrame(self.trades)
    
    def get_equity_curve_df(self):
        return pd.DataFrame(self.equity_curve)
    
def run_backtest(model, dataloader, device='cuda' if torch.cuda.is_available() else 'cpu'):
    model.to(device)
    model.eval()
    
    strategy = TradingStrategy()
    
    with torch.no_grad():
        loop = tqdm(dataloader, desc='Backtesting')
        for features, times, datetime, close_price in loop:
            features = features.to(device)
            times = times.to(device)
            
            # Get model prediction
            outputs = model(features, times)
            probs = torch.softmax(outputs, dim=-1)
            prediction = torch.argmax(probs, dim=-1).item()
            confidence = probs[0][prediction].item()
            
            # Check stop loss / take profit for existing position
            strategy.check_stop_loss_take_profit(close_price.item(), datetime.item())
            
            # Open new position if confidence is high enough
            if confidence > 0.6:  # Confidence threshold
                if prediction == 0 and strategy.current_position is None:  # BUY
                    strategy.open_position('BUY', close_price.item(), datetime.item())
                elif prediction == 1 and strategy.current_position is None:  # SELL
                    strategy.open_position('SELL', close_price.item(), datetime.item())
                elif prediction == 2 and strategy.current_position is not None:  # HOLD -> Close position
                    strategy.close_position(close_price.item(), datetime.item())
            
            # Update equity curve
            strategy.update_equity(close_price.item(), datetime.item())
            
    return strategy

def plot_equity_curve(equity_df):
    plt.figure(figsize=(15, 6))
    plt.plot(equity_df['datetime'], equity_df['equity'])
    plt.title('Equity Curve')
    plt.xlabel('Date')
    plt.ylabel('Equity')
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

def analyze_trades(trades_df):
    if trades_df.empty:
        print("No trades were executed.")
        return
    
    # Filter only exit trades
    exit_trades = trades_df[trades_df['type'].isin(['EXIT', 'STOP_LOSS', 'TAKE_PROFIT'])]
    
    # Calculate metrics
    total_trades = len(exit_trades)
    profitable_trades = len(exit_trades[exit_trades['pnl'] > 0])
    win_rate = profitable_trades / total_trades if total_trades > 0 else 0
    
    total_profit = exit_trades['pnl'].sum()
    max_drawdown = min(exit_trades['pnl'])
    
    avg_profit = exit_trades[exit_trades['pnl'] > 0]['pnl'].mean()
    avg_loss = exit_trades[exit_trades['pnl'] < 0]['pnl'].mean()
    
    profit_factor = abs(exit_trades[exit_trades['pnl'] > 0]['pnl'].sum() / exit_trades[exit_trades['pnl'] < 0]['pnl'].sum()) if len(exit_trades[exit_trades['pnl'] < 0]) > 0 else float('inf')
    
    # Print results
    print("\n=== Trading Performance Analysis ===")
    print(f"Total Trades: {total_trades}")
    print(f"Profitable Trades: {profitable_trades}")
    print(f"Win Rate: {win_rate:.2%}")
    print(f"Total Profit: ${total_profit:.2f}")
    print(f"Maximum Drawdown: ${max_drawdown:.2f}")
    print(f"Average Profit: ${avg_profit:.2f}")
    print(f"Average Loss: ${avg_loss:.2f}")
    print(f"Profit Factor: {profit_factor:.2f}")
    
    # Plot trade distribution
    plt.figure(figsize=(10, 6))
    plt.hist(exit_trades['pnl'], bins=50)
    plt.title('Trade PnL Distribution')
    plt.xlabel('Profit/Loss ($)')
    plt.ylabel('Frequency')
    plt.grid(True)
    plt.show()


In [10]:
# Load and preprocess test data
test_df = load_data('data/xauusd/1m/dynamic_labeled_test.csv')  # Replace with your test data file
test_df = add_technical_indicators(test_df)
test_df = add_basic_features(test_df)

# Define feature columns
feature_cols = [col for col in test_df.columns if col not in ['Label', 'Datetime']]

# Create dataset and dataloader
SEQ_LEN = 128
BATCH_SIZE = 1  # Use batch size 1 for backtesting
test_dataset = BacktestDataset(test_df, SEQ_LEN, feature_cols, use_time2vec=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Load model
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
N_FEATURES = len(feature_cols)
model = EnhancedHybridModel(
    num_features=N_FEATURES, 
    time_dim=1, 
    num_classes=3, 
    d_model=128, 
    nhead=8, 
    dim_feedforward=256, 
    num_layers=3
)

# Load trained model weights
model.load_state_dict(torch.load('save/models/used_model.pth', map_location=DEVICE))

# Run backtest
strategy = run_backtest(model, test_loader, DEVICE)

# Analyze results
trades_df = strategy.get_trades_df()
equity_df = strategy.get_equity_curve_df()

# Plot results
plot_equity_curve(equity_df)
analyze_trades(trades_df)

# Save results
trades_df.to_csv('backtest_trades.csv', index=False)
equity_df.to_csv('backtest_equity.csv', index=False)


Adding technical indicators...
Technical indicators added.
Applying preprocessing...
Calculating percentage change for: ['Open', 'High', 'Low', 'Close']
Adding time features...
Preprocessing completed.


C:\Users\longt\AppData\Local\Temp\ipykernel_19208\776879440.py:29: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('save/models/used_model.pth

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'numpy.datetime64'>